In [75]:
from sklearn import datasets
import pandas as pd
import numpy as np
# import some data to play with
iris = datasets.load_iris()
import random
import os
from sklearn.preprocessing import LabelEncoder

In [76]:
def get_test_data_single_col():
    """Creates test dataframes and imputes the missing values"""
    test_set = []
    for size in range(20,100,10):
        col_names = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')[:10]
        rand_col = random.choice(col_names)
        curr_df = pd.DataFrame(np.random.randint(-50,100,size=(size,10)), columns=col_names)
        curr_df.loc[curr_df.sample(np.random.randint(5,int(size*0.4)), random_state=32).index,rand_col] = np.nan
        test_set.append((curr_df,rand_col,curr_df[rand_col].mean(skipna=True)))

    # for i in range(test_n):
    #     test_set.append((np.array(test_vectors[i,:],dtype='complex'),np.fft.fft(test_vectors[i,:]).reshape((-1,1))))
    # return [(np.array(test_vectors[0,:],dtype='complex'),np.fft.fft(test_vectors[0,:]).reshape((-1,1)))]
    return test_set

In [77]:
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_percentage_error,mean_squared_error
import pandas as pd
import numpy as np
import random

# x = get_test_data_single_col()
# df = x[0][0]
# curr_df = df.dropna(axis=0,how='any').copy()
# path_project = os.path.abspath(
# os.path.join(__file__, "../.."))
# path_processed_data = +'/data/processed/'
# path_raw_data = path_project+'/data/raw/'
curr_df = pd.read_csv('..\data\processed\iris_original.csv')

class MySimpleImputer():
    def __init__(self,target_col,**kwargs):
        self.target_col = target_col
        self.strategy = kwargs.get('strategy','mean')

    def impute(self,df):
        imp = SimpleImputer(missing_values=np.nan, strategy=self.strategy)
        if self.target_col == 'ALL':
            idf = pd.DataFrame(imp.fit_transform(df))
            idf.columns = df.columns
            idf.index = df.index
        else:
            idf = df.copy()
            imp.fit(idf[self.target_col].values.reshape(-1, 1))
            idf[self.target_col] = imp.transform(idf[self.target_col].values.reshape(-1, 1))
        return idf
imputer = MySimpleImputer(target_col='sepal length (cm)')        
nans = curr_df.mask((np.random.random(curr_df.shape)<.4) & (curr_df.columns == imputer.target_col))
target = curr_df[nans[imputer.target_col].isna()][imputer.target_col].values
res = imputer.impute(nans)
res_values = res[nans[imputer.target_col].isna()][imputer.target_col].values
mean_squared_error(target,res_values,squared=True)

0.717742857142857

In [78]:
def replace_infs(df):
    """Replaces infinities with min/max of the column

    Parameters
    ----------
    df : pd.DataFrame
        Input data

    Returns
    -------
    pd.DataFrame
    """
    m1 = df.eq(np.inf)
    m2 = df.eq(-np.inf)
    df = df.mask(m1, df[~m1].max(), axis=1).mask(m2, df[~m2].min(), axis=1)
    return df

In [79]:
nans = curr_df.mask((np.random.random(curr_df.shape)<.4))
imputer = MySimpleImputer(target_col='ALL')
res = imputer.impute(nans)

In [80]:
curr_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [115]:
curr_df['target'] = curr_df['target']
le = LabelEncoder()
curr_df['target'] = le.fit_transform(curr_df['target'])
# Dummify rest categorical cols
df_target_col = pd.concat([curr_df.drop('target',axis=1), pd.get_dummies(curr_df['target']).astype('category')], axis=1)

In [116]:
df_target_col.dtypes

sepal length (cm)     float64
sepal width (cm)      float64
petal length (cm)     float64
petal width (cm)      float64
0                    category
1                    category
2                    category
dtype: object

In [61]:
# x = pd.DataFrame(((res-curr_df)**2).mean(axis=0)**(1/curr_df.shape[0])).T
# y = pd.DataFrame(((res-curr_df)**2).mean(axis=0)**(1/curr_df.shape[0])).T
# x = pd.concat([x,y],axis=0)
# x

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,0.990229,0.982495,1.001646,0.988177,0.991107
0,0.990229,0.982495,1.001646,0.988177,0.991107


0.9902286360659346
0.9824953305016363
1.0016456411186456
0.9881767246283486
0.9911072841907653
